In [1]:
# IMPORTS

# Base
import json
import pandas as pd

# Inner
from scripts.file_manager import YAML_read, JSON_load, JSON_open
from scripts.requests import Request_post

# Old Inner Libs
from scripts.googlesheets_upload_lib import ListConventer
from scripts.intervals_api import *


In [2]:
# Get settings
file = YAML_read('config')
file.get_config()
token = file.config['service_token'] # Exclude API token
link = file.config['service_link'] # Exclude API url
json_o = JSON_open('data')
json_o.open()
start_data = json_o.data
actual_data_id = start_data[0]['id'] # This is maximum data id in offline dataset

In [3]:
# Get data on cycle request 

result = Request_post(link, token) # First request
data = []
snap = result.result
num = 1
work = True
while work:
    if snap['next']:
        for result in snap['results']:
            if result['id'] > actual_data_id:
                data += [result]
                link = snap['next']
                result = Request_post(link, token)
                snap = result.result
                num += 1
            else:
                print('Actual data not found')
                work = False
                break
        
    else:
        work = False


Actual data not found


In [4]:
# Save data
data = data+start_data
json_l = JSON_load('data')
json_l = json_l.load(data)

In [5]:
# Pandas
df=pd.DataFrame(data)
ids = list(df['identity'].unique())
users_count = len(df['identity'].unique())
print(f'Количество пользователей - {users_count}')
df['date'] = df.apply(lambda x: x['time_create'].split('T')[0],axis=1)
df.head()

Количество пользователей - 210


,id,identity,command,time_create,date
0,2143,315854463,progress_words,2022-12-21T18:32:39.305321+03:00,2022-12-21
1,2142,409308393,open_settings,2022-12-21T15:13:22.662603+03:00,2022-12-21
2,2141,315854463,exit,2022-12-21T15:04:03.171655+03:00,2022-12-21
3,2140,315854463,new_words,2022-12-21T15:03:55.157946+03:00,2022-12-21
4,2139,315854463,exit,2022-12-21T15:03:53.964027+03:00,2022-12-21


In [6]:
def intervals_link(id):
    return f'https://intervals.ru/data/api/v1/{str(id)}/'
# cr
gic = GetIntervalsConfig()
gic.get_token('config')

In [7]:
def get_days(row):
    try:
        return len(list(df[df['identity'] == row['identity']]['date'].unique()))
    except:
        0
        
def get_counts(row):
    return df['identity'].value_counts ()[row['identity']]

# Data days, counts
days_stat = pd.DataFrame(ids, columns=['identity'])
days_stat['days'] = days_stat.apply(get_days, axis=1)
days_stat['counts'] = days_stat.apply(get_counts, axis=1)
days_stat.sort_values(by='days', ascending=False).reset_index()[['identity', 'days', 'counts']].head(10)

# Upload data on intervals
lc = ListConventer(days_stat,'width_headers')
lc.conventer()
gic = GetIntervalsConfig()
gic.get_token('config')
IntervalsUpload(gic.token, intervals_link(168), lc.datsheets)   

Данные АПИ получены


/home/ustsl/Рабочий стол/datascience projects/bilmek_stat/scripts/intervals_api.py:19: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = load(f)


Данные успешно загружены в контейнер
<Response [200]>


In [8]:
# Data commands
commands = df['command'].value_counts().reset_index()
commands.columns = ['command', 'counts']


# Upload data on intervals
lc = ListConventer(commands.head(30),'width_headers')
lc.conventer()

IntervalsUpload(gic.token, intervals_link(169), lc.datsheets)   

,command,counts
0,exit,357
1,new_words,330
2,/start,256
3,/menu,145
4,open_settings,111
5,repeat_words,110
6,grammar_book,103
7,proceed_start,89
8,main_menu,73
9,progress_words,69
